In [1]:
import pandas as pd
df = pd.read_csv('../Data/lstm.csv')
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [2]:
df.category.unique()

array(['food', 'sports'], dtype=object)

In [3]:
# 데이터에 사용된 중복 없는 전체 단어 갯수를 파악
result = set()
df['paragraph'].str.lower().str.split().apply(result.update)
vocab_size = len(result)
vocab_size

536

In [5]:
# 단어를 숫자로 인코딩
paragraphs = df['paragraph'].to_list() # 길이가 다른 건 numpy를 못쓴다.
paragraphs[0:5]

['dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait',
 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations',
 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table',
 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition',
 'the biscuits and gravy was too salty two people in my group had the gravy and all thoug

In [6]:
from tensorflow import keras
keras.utils.set_random_seed(1) # keras set seed

In [7]:
# one hot encoding
encoded_paragraphs = [keras.preprocessing.text.one_hot(paragraph , vocab_size) for paragraph in paragraphs]
encoded_paragraphs

[[25,
  532,
  236,
  181,
  22,
  288,
  289,
  532,
  303,
  99,
  205,
  173,
  277,
  156,
  171,
  528,
  116,
  444,
  531,
  61,
  408,
  105,
  432,
  488,
  273,
  179,
  402,
  174,
  444,
  256,
  185,
  422,
  447,
  313,
  139,
  232,
  422,
  142,
  465,
  413,
  522,
  402,
  202,
  302,
  479,
  351,
  13,
  329,
  515,
  205,
  321,
  444,
  45],
 [146,
  171,
  528,
  325,
  61,
  413,
  432,
  277,
  399,
  444,
  524,
  336,
  179,
  461,
  312,
  493,
  398,
  356,
  447],
 [345,
  54,
  75,
  186,
  444,
  8,
  179,
  182,
  202,
  320,
  54,
  465,
  497,
  182,
  444,
  8,
  532,
  134,
  164,
  55,
  378,
  286,
  14,
  59,
  339,
  398,
  185,
  444,
  17,
  495,
  329,
  54,
  172,
  179,
  472,
  35,
  444,
  45,
  164,
  349,
  422,
  139,
  368],
 [334,
  484,
  197,
  429,
  244,
  205,
  152,
  179,
  429,
  364,
  179,
  152,
  33,
  205,
  76,
  142,
  358,
  179,
  205,
  33,
  142,
  497,
  431,
  260,
  311,
  198,
  205,
  33,
  390,
  515,
  179,


In [9]:
# 데이터에서 가장 긴 문장의 단어 갯수 확인 -> padding 주기 위함
max_length = 0
for row in df['paragraph']:
    if len(row.split(" ")) > max_length:
        max_length = len(row.split(" "))
print(max_length)

91


In [10]:
# 문장마다 단어 갯수가 다르므로 sequence padding을 넣어서 문장의 길이를 동일하게 만듭니다.
padded_paragraph_encoding = keras.preprocessing.sequence.pad_sequences(
    encoded_paragraphs , maxlen = max_length , padding='post' ) # padding='post' : 0값이 뒤에 들어간다

padded_paragraph_encoding

array([[ 25, 532, 236, ...,   0,   0,   0],
       [146, 171, 528, ...,   0,   0,   0],
       [345,  54,  75, ...,   0,   0,   0],
       ...,
       [459, 488, 422, ...,   0,   0,   0],
       [277, 205, 301, ...,   0,   0,   0],
       [523, 174, 205, ...,   0,   0,   0]], dtype=int32)

In [11]:
# 분류항목(food , sport)를 수치로 변형하기
categories = df['category'].to_list()

def category_encode(category):
    if category == 'food':
        return [1,0]
    else:
        return [0,1]

In [12]:
encoded_category = [category_encode(category) for category in categories]
encoded_category

[[1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1]]

In [13]:
# Feature 확인
print(encoded_paragraphs[0])

[25, 532, 236, 181, 22, 288, 289, 532, 303, 99, 205, 173, 277, 156, 171, 528, 116, 444, 531, 61, 408, 105, 432, 488, 273, 179, 402, 174, 444, 256, 185, 422, 447, 313, 139, 232, 422, 142, 465, 413, 522, 402, 202, 302, 479, 351, 13, 329, 515, 205, 321, 444, 45]


In [14]:
len(encoded_paragraphs[19])

73

---
### 주제를 분류하는 모델 구현하기

In [15]:
model = keras.Sequential()
# 모델 생성 단계
model.add(keras.layers.Embedding(vocab_size , 5 , input_length = max_length)) 
# 임베딩 레이어는 인덱스를 받아 5차원 벡터의 임베딩을 출력합니다. -> 영어 문법 (영어 5형식) (주어, 동사, 목적어, 보어, 수식어)
# 3부터 정확도가 올라감
model.add(keras.layers.LSTM(64)) 

# 분류 단계 
model.add(keras.layers.Dense(32 , activation = "relu")) # LSTM의 반값 사용
model.add(keras.layers.Dense(2, activation = "softmax")) # 2개 내놓고 확률을 보기위함

2022-07-22 14:20:04.207769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
model.compile(loss="categorical_crossentropy" , optimizer="adam" , metrics= 'accuracy')

In [18]:
import numpy as np
train_X = np.array(padded_paragraph_encoding)
train_y = np.array(encoded_category)

In [19]:
model.fit(train_X , train_y , batch_size= 10 , epochs=50)
# model.fit(train_X , train_y , epochs=50) # mini_batch = 1 , default 1개씩 들어간것

Epoch 1/50
2/2 [==============================] - 2s 17ms/step - loss: 0.6940 - accuracy: 0.5500
Epoch 2/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6927 - accuracy: 0.5000
Epoch 3/50
2/2 [==============================] - 0s 21ms/step - loss: 0.6926 - accuracy: 0.5500
Epoch 4/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6927 - accuracy: 0.6000
Epoch 5/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6924 - accuracy: 0.5500
Epoch 6/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6920 - accuracy: 0.5000
Epoch 7/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6916 - accuracy: 0.6000
Epoch 8/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6912 - accuracy: 0.6000
Epoch 9/50
2/2 [==============================] - 0s 21ms/step - loss: 0.6906 - accuracy: 0.6000
Epoch 10/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6902 - accuracy: 0.6000
Epoch 11/50
2/2 [============

---